In [3]:
import memcache
import random
import time
import tkinter as tk
from threading import Thread


In [ ]:

# Connect to the Memcached server
mc = memcache.Client(['XXXX'], debug=0)

# Función para agregar o actualizar un puntaje en el leaderboard
def add_score(name, score):
    leaderboard = mc.get('leaderboard') or {}
    leaderboard[name] = score
    mc.set('leaderboard', leaderboard)

    # Actualizar y guardar historial
    historial_entrada = f"{time.ctime()}: {score}"
    historial = client.get(f"{name}_historial") or []
    historial.append(historial_entrada)
    client.set(f"{name}_historial", historial)

# Función para recuperar los 50 mejores puntajes del leaderboard
def get_leaderboard():
    leaderboard = mc.get('leaderboard') or {}
    # Filtrar para incluir solo los nombres en la lista de nicknames
    filtered_leaderboard = {name: score for name, score in leaderboard.items() if name in nicknames}
    sorted_leaderboard = sorted(filtered_leaderboard.items(), key=lambda x: x[1], reverse=True)[:50]
    return sorted_leaderboard

# Función para simular actualizaciones en tiempo real
def simulate_live_updates(names, max_updates=10):
    updates = 0
    while updates < max_updates:
        leaderboard = mc.get('leaderboard') or {}
        for name in names:
            current_score = leaderboard.get(name, 0)
            new_score = random.randint(current_score + 1, current_score + 50)
            add_score(name, new_score)

        update_leaderboard_display()  # Actualiza la visualización del leaderboard
        time.sleep(10)  # Esperar 10 segundos antes de la siguiente actualización
        updates += 1

# Función para mostrar el historial de un jugador
def mostrar_historial(jugador_nombre):
    historial = client.get(f"{jugador_nombre}_historial") or []
    historial_text = f"Historial de {jugador_nombre}:\n"
    for entrada in historial:
        historial_text += f"{entrada}\n"
    return historial_text


# Función para actualizar la visualización del leaderboard en la interfaz
def update_leaderboard_display():
    top_players = get_leaderboard()
    leaderboard_text = "Current Leaderboard:\n"
    if top_players:
        for player, score in top_players:
            leaderboard_text += f'{player}: {score}\n'
    else:
        leaderboard_text = "Leaderboard is empty"
    
    leaderboard_label.config(text=leaderboard_text)

# Función para iniciar la simulación en un hilo
def start_simulation():
    simulate_live_updates(nicknames, max_updates=5)

# Crear la ventana principal de Tkinter
root = tk.Tk()
root.title("Leaderboard")

# Etiqueta de título
title_label = tk.Label(root, text="Leaderboard", font=("Arial", 16))
title_label.pack(pady=10)

# Etiqueta para mostrar el leaderboard
leaderboard_label = tk.Label(root, text="", font=("Arial", 12), justify="left")
leaderboard_label.pack(padx=10, pady=10)

# Botón para iniciar la simulación
start_button = tk.Button(root, text="Iniciar Simulación", command=lambda: Thread(target=start_simulation).start())
start_button.pack(pady=20)

# Lista de nicknames personalizada (50 jugadores)
nicknames = ['DragonSlayer', 'ShadowHunter', 'StarGazer', 'MysticWarrior', 'SpeedDemon',
             'ThunderStrike', 'IronFist', 'NightRider', 'FireMage', 'StormBringer',
             'IceQueen', 'DarkKnight', 'WindWalker', 'SkyRuler', 'BladeMaster',
             'StoneGolem', 'SolarFlare', 'Moonlight', 'CrimsonBlade', 'SilverFox',
             'GoldenLion', 'Phantom', 'SilentAssassin', 'GhostRider', 'Inferno',
             'OceanWave', 'BlackPanther', 'FalconEye', 'Rogue', 'Viper', 
             'WolfSpirit', 'WhiteTiger', 'SteelHawk', 'CobraKing', 'Titan', 
             'Hawkeye', 'StormShadow', 'TigerClaw', 'Lightning', 'Venom',
             'Firestorm', 'CrystalPhoenix', 'Warlock', 'Shaman', 'Necromancer', 
             'Berserker', 'Samurai', 'Valkyrie', 'Ranger', 'Paladin']

# Inicializa con puntajes aleatorios
for nickname in nicknames:
    add_score(nickname, random.randint(100, 6000))

# Ejecutar el bucle principal
root.mainloop()
